In [ ]:
!pip install mysql-connector-python 

In [ ]:
!pip install tabulate


In [ ]:
pip install sqlalchemy pymysql

In [ ]:
mycursor.execute("CREATE DATABASE proj_1")

In [ ]:
import pandas as pd


In [ ]:
import mysql.connector

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
import json

In [ ]:
mydb=mysql.connector.connect(
    host="localhost",
    username="root",
    password="",
    port=3306
)

In [ ]:
mycursor=mydb.cursor(buffered=True)

In [ ]:
from tabulate import tabulate

In [ ]:
mycursor.execute("USE proj_1")

In [ ]:
username = 'root'
password = 'your_password'
host = 'localhost'
port = 3306
database = 'proj_1'

engine = create_engine('mysql+pymysql://root@localhost/proj_1')


In [ ]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=F9HPG6aTDbVhgSSPpmNfkY5tLzKwOAdbcjU9u23T"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

In [ ]:
data=json.loads(response.text)

In [ ]:
data

In [ ]:
competitions=data["competitions"]
competitions

In [ ]:
categories=[]
for x in competitions:
    categories.append({
        "category_id":x["category"]["id"],
        "category_name":x["category"]["name"]
    })

In [ ]:
a=pd.DataFrame(categories)
a

In [ ]:
competitions_table=[]
for x in competitions:
    competitions_table.append({
        "competition_id":x["id"],
        "competition_name":x["name"],
        "parent_id":x.get("parent_id"),
        "type":x["type"],
        "gender":x["gender"],
        
        "category_id":x["category"]["id"],
        
            
        })
    
        

In [ ]:
b=pd.DataFrame(competitions_table)
b

In [ ]:
a.to_sql(name='category_table', con=engine, if_exists='replace', index=False)

In [ ]:
b.to_sql(name='competition_table', con=engine, if_exists='replace', index=False)

In [ ]:
mycursor.execute("ALTER TABLE category_table MODIFY COLUMN category_id VARCHAR(50) PRIMARY KEY")

In [ ]:
mycursor.execute("SELECT category_id, COUNT(*) AS count FROM category_table GROUP BY category_id HAVING count > 1")

out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
mycursor.execute("CREATE TABLE temp_table AS SELECT DISTINCT * FROM category_table")

In [ ]:
 mycursor.execute("DELETE FROM category_table")

In [ ]:
mycursor.execute("INSERT INTO category_table (category_id, category_name) SELECT category_id, category_name FROM temp_table")

In [ ]:
mycursor.execute("DROP TABLE temp_table")

In [ ]:
mycursor.execute("ALTER TABLE category_table MODIFY COLUMN category_id VARCHAR(50) PRIMARY KEY")

In [ ]:
mycursor.execute("SELECT * FROM category_table")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


In [ ]:
mycursor.execute("ALTER TABLE category_table MODIFY category_id VARCHAR(50)")

In [ ]:
 mycursor.execute("ALTER TABLE category_table MODIFY category_name VARCHAR(100) NOT NULL")

In [ ]:
 mycursor.execute("ALTER TABLE competition_table MODIFY category_id VARCHAR(50)")

In [ ]:
 mycursor.execute("ALTER TABLE competition_table MODIFY competition_id VARCHAR(50)")

In [ ]:
mycursor.execute("ALTER TABLE competition_table MODIFY competition_name VARCHAR(100) NOT NULL")

In [ ]:
 mycursor.execute("ALTER TABLE competition_table ADD PRIMARY KEY (competition_id (50))")

In [ ]:
 mycursor.execute("ALTER TABLE competition_table MODIFY parent_id VARCHAR(50) NOT NULL")

In [ ]:
 mycursor.execute("ALTER TABLE competition_table MODIFY type VARCHAR(20) NOT NULL")

In [ ]:
 mycursor.execute("ALTER TABLE competition_table MODIFY gender VARCHAR(10) NOT NULL")

In [ ]:
mycursor.execute("ALTER TABLE competition_table ADD CONSTRAINT forkeya FOREIGN KEY (category_id) REFERENCES category_table(category_id)")


In [ ]:
mycursor.execute("SELECT *, COUNT(*)FROM competition_table GROUP BY competition_id HAVING COUNT(*) > 1")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# 1.List all competitions along with their category name
mycursor.execute("""SELECT competition_table.competition_id, competition_table.competition_name, category_table.category_name 
                 FROM competition_table
                 JOIN category_table ON category_table.category_id=competition_table.category_id""")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))    

In [ ]:
# 2.Count the number of competitions in each category
mycursor.execute("SELECT category_id, COUNT(*) AS competition_count FROM competition_table GROUP BY category_id")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# 3.Find all competitions of type 'doubles
mycursor.execute("SELECT competition_id, competition_name,type FROM competition_table WHERE type='doubles'")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# 4.Get competitions that belong to a specific category (e.g., ITF Men)
specific_category=input("Enter category name:")
query="""
    SELECT category_name, competition_id,competition_name
    FROM competition_table comp
    JOIN category_table cat ON comp.category_id = cat.category_id
    WHERE cat.category_name = %s"""
mycursor.execute(query,(specific_category,))
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

In [ ]:
# 5.Identify parent competitions and their sub-competitions
mycursor.execute("""
    SELECT category_table.category_id AS parent_competitions ,competition_table.competition_id AS sub_competitions
    FROM competition_table
    JOIN category_table ON competition_table.category_id = category_table.category_id
""")

out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


In [ ]:
# 6.Analyze the distribution of competition types by category
mycursor.execute("""SELECT category_table.category_id,competition_table.competition_name, competition_table.type
                 FROM category_table 
                 JOIN competition_table ON category_table.category_id=competition_table.category_id
                 
""")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#7.List all competitions with no parent (top-level competitions)
mycursor.execute("""
    SELECT competition_id,competition_name,parent_id
    FROM competition_table 
    WHERE parent_id = ''
""")

out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))


In [ ]:
import requests
url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=F9HPG6aTDbVhgSSPpmNfkY5tLzKwOAdbcjU9u23T"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)
print(response.text)

In [ ]:

data2=json.loads(response.text)
data2

In [ ]:
complexes=data2["complexes"]
complexes

In [ ]:
complexes_table=[]
for x in complexes:
    complexes_table.append({
        "complex_id":x["id"],
        "complex_name":x["name"]
    })
c=pd.DataFrame(complexes_table)
c

In [ ]:
venue_table = []

for x in complexes:
    venues = x.get("venues",[]) 
    for venue in venues:
        venue_table.append({
            "venue_id": venue.get("id"),
            "venue_name": venue.get("name"),
            "city_name": venue.get("city_name"),
            "country_name": venue.get("country_name"),
            "country_code": venue.get("country_code"),
            "timezone": venue.get("timezone"),
            "complex_id": x.get("id") 
        })

d=pd.DataFrame(venue_table)
d

In [ ]:
c.to_sql(name='complexes_table', con=engine, if_exists='append', index=False)

In [ ]:
d.to_sql(name='venues_table', con=engine, if_exists='append', index=False)

In [ ]:
mycursor.execute("ALTER TABLE complexes_table MODIFY COLUMN complex_id VARCHAR(50)")


In [ ]:
mycursor.execute("ALTER TABLE complexes_table MODIFY COLUMN complex_name VARCHAR(100) NOT NULL;")


In [ ]:
mycursor.execute("ALTER TABLE complexes_table ADD PRIMARY KEY (complex_id);")

In [ ]:
mycursor.execute("ALTER TABLE venues_table MODIFY COLUMN venue_id VARCHAR(50) PRIMARY KEY;")

In [ ]:
mycursor.execute("ALTER TABLE venues_table MODIFY COLUMN venue_name VARCHAR(100) NOT NULL;")

In [ ]:
mycursor.execute("ALTER TABLE venues_table MODIFY COLUMN city_name VARCHAR(100) NOT NULL;")

In [ ]:
mycursor.execute("ALTER TABLE venues_table MODIFY COLUMN country_name VARCHAR(100) NOT NULL;")

In [ ]:
mycursor.execute("ALTER TABLE venues_table MODIFY COLUMN country_code CHAR(3) NOT NULL;")

In [ ]:
mycursor.execute("ALTER TABLE venues_table MODIFY COLUMN timezone VARCHAR(100) NOT NULL;")

In [ ]:
mycursor.execute("ALTER TABLE venues_table MODIFY COLUMN complex_id VARCHAR(50)")

In [ ]:
mycursor.execute("ALTER TABLE venues_table ADD CONSTRAINT forkeyb FOREIGN KEY (complex_id) REFERENCES complexes_table (complex_id)")



In [ ]:
# 1.List all venues along with their associated complex name
mycursor.execute("SELECT venues_table.venue_name, complexes_table.complex_name FROM venues_table JOIN complexes_table")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:

# 2.Count the number of venues in each complex
mycursor.execute("SELECT complex_id, count(venue_name) AS venue_count FROM venues_table GROUP BY complex_id")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# 3.Get details of venues in a specific country (e.g., Chile)
country=input("Enter country name:")
query="""
SELECT country_name ,venue_id, venue_name, city_name FROM venues_table WHERE country_name= %s"""
mycursor.execute(query,(country,))
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# 4.Identify all venues and their timezones
mycursor.execute("SELECT venue_id, venue_name ,timezone FROM venues_table")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# 5.Find complexes that have more than one venue
mycursor.execute("SELECT complex_id, COUNT(venue_id) AS venue_count FROM venues_table GROUP BY complex_id HAVING COUNT(venue_id) >1")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# 6.List venues grouped by country
mycursor.execute("SELECT country_name,venue_name, COUNT(venue_name) FROM venues_table GROUP BY country_name")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


In [ ]:

# 7.Find all venues for a specific complex (e.g., Nacional)
complex=input("Enter complex name:")
query="""
SELECT complexes_table.complex_name, venues_table.venue_name 
FROM venues_table 
JOIN complexes_table ON complexes_table.complex_id=venues_table.complex_id
WHERE complexes_table.complex_name=%s"""
mycursor.execute(query,(complex,))
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
import requests
url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=F9HPG6aTDbVhgSSPpmNfkY5tLzKwOAdbcjU9u23T"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)
print(response.text)

In [ ]:
data3=json.loads(response.text)
data3

In [ ]:
rankings=data3["rankings"]
rankings

In [ ]:
rankings_table=[]
for x in rankings:
    competitor_rankings=x.get("competitor_rankings",{})
    for y in competitor_rankings:
        rankings_table.append({
            
            "rank":y["rank"],
            "movement":y["movement"],
            "points":y["points"],
            "competitions_played":y["competitions_played"],
            "competitor_id":y["competitor"]["id"]
    })
       
e=pd.DataFrame(rankings_table)
e


In [ ]:
competitor_table=[]
for x in rankings:
              competitor_rankings = x.get("competitor_rankings", [])
              for y in competitor_rankings:
                      competitor=y.get("competitor",[])
                      for z in competitor:
                                       competitor_table.append({
                                      "competitor_id":competitor["id"],
                                      "name":competitor["name"],            
                                      "country":competitor["country"],
                                      "country_code":competitor.get("country_code"),
                                      "abbreviation":competitor["abbreviation"]
                                       })
f=pd.DataFrame(competitor_table)
f

In [ ]:
e.to_sql(name='rankings_table', con=engine, if_exists='append', index=False)


In [ ]:
f.to_sql(name='competitor_table', con=engine, if_exists='append', index=False)

In [ ]:
mycursor.execute("ALTER TABLE rankings_table ADD rank_id INT PRIMARY KEY AUTO_INCREMENT")

In [ ]:
mycursor.execute("ALTER TABLE rankings_table MODIFY COLUMN rank INT NOT NULL")


In [ ]:
mycursor.execute("ALTER TABLE rankings_table MODIFY COLUMN movement INT NOT NULL")


In [ ]:
mycursor.execute("ALTER TABLE rankings_table MODIFY COLUMN points INT NOT NULL")


In [ ]:
mycursor.execute("ALTER TABLE rankings_table MODIFY COLUMN competitions_played INT NOT NULL")



In [ ]:
mycursor.execute("ALTER TABLE rankings_table MODIFY COLUMN competitor_id VARCHAR(50)")



In [ ]:
mycursor.execute("ALTER TABLE rankings_table ADD CONSTRAINT forkeyc FOREIGN KEY (competitor_id) REFERENCES competitor_table(competitor_id)")

In [ ]:
mycursor.execute("ALTER TABLE competitor_table MODIFY COLUMN competitor_id VARCHAR(50) PRIMARY KEY")

In [ ]:
mycursor.execute("SELECT COUNT(DISTINCT competitor_id) FROM competitor_table")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
mycursor.execute("CREATE TABLE temp_table LIKE competitor_table ")

In [ ]:
mycursor.execute("INSERT INTO temp_table SELECT DISTINCT * FROM competitor_table")

In [ ]:
mycursor.execute("DELETE FROM competitor_table")

In [ ]:

mycursor.execute("INSERT INTO competitor_table SELECT * FROM temp_table")

In [ ]:
mycursor.execute("DROP TABLE temp_table")

In [ ]:
mycursor.execute("ALTER TABLE competitor_table MODIFY COLUMN competitor_id VARCHAR(50) PRIMARY KEY")

In [ ]:
mycursor.execute("ALTER TABLE competitor_table MODIFY COLUMN name VARCHAR(100)NOT NULL")


In [ ]:
mycursor.execute("ALTER TABLE competitor_table MODIFY COLUMN country VARCHAR(100)NOT NULL")


In [ ]:
mycursor.execute("ALTER TABLE competitor_table MODIFY COLUMN country_code CHAR(3)NOT NULL")


In [ ]:
mycursor.execute("ALTER TABLE competitor_table MODIFY COLUMN abbreviation VARCHAR(10)NOT NULL")

In [ ]:
# 1.Get all competitors with their rank and points.
mycursor.execute("SELECT competitor_id, rank, points FROM rankings_table ")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# 2.Find competitors ranked in the top 5
mycursor.execute("SELECT competitor_id, rank FROM rankings_table ORDER BY rank LIMIT 5 ")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql')) 


In [ ]:
# 3.List competitors with no rank movement (stable rank)
mycursor.execute("""
    SELECT competitor_id,movement
    FROM rankings_table
    WHERE movement = 0
""")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# 4.Get the total points of competitors from a specific country (e.g., Croatia)
country = input("Enter country name: ")
query = """
    SELECT c.country, SUM(r.points) AS total_points
    FROM rankings_table r
    JOIN competitor_table c ON r.competitor_id = c.competitor_id
    WHERE c.country = %s
    GROUP BY c.country
"""
mycursor.execute(query, (country,))
out = mycursor.fetchall()

print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))


In [ ]:
# 5.Count the number of competitors per country
mycursor.execute("""SELECT country, COUNT(competitor_id) as competitor_count
                 FROM competitor_table
                 GROUP BY country
                 ORDER BY competitor_count """)
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

In [ ]:
# 6.Find competitors with the highest points in the current week
mycursor.execute("""SELECT competitor_table.competitor_id, competitor_table.name, rankings_table.points
                 FROM competitor_table
                 JOIN rankings_table ON competitor_table.competitor_id=rankings_table.competitor_id """)
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))